# Will be using:
 1. Langchain framework for RAG Architecture.
 2. Faiss for Vector DB
 3. PyPDF for pdf text extraction.
 4. sentence transformer for vector embedding.
 5. Free Chatgpt API from RAPID API.

In [1]:
!pip install langchain
!pip install faiss-gpu
!pip install pypdf
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.8 MB/s eta 0:00:00
  Using cached nvidia_cuda

In [2]:
from langchain.schema.output_parser import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.vectorstores.utils import filter_complex_metadata
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [3]:
from google.colab import files
uploaded = files.upload()
# Uploading the file named Doc1.pdf
txt_file_path = 'Doc1.pdf'

Saving Doc1.pdf to Doc1.pdf


In [4]:
#Extracting the text and splitting into chunks to be vectorized.
docs = PyPDFLoader(file_path=txt_file_path).load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=80)
chunks = text_splitter.split_documents(docs)
chunks = filter_complex_metadata(chunks)
#Using the embedding model named all-MiniLM-L6-v2, more can be read in HuggingFace
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
#Storing the chunks in vector DB
db = FAISS.from_documents(documents=chunks, embedding=embedding_function)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
#Moding the retrival algorithm to pick particular number of chunks after it crosses the threshold.
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "score_threshold": 0.4,
        "k":3},
)

## This question can be found on the 6th page in the first line of the Doc1.pdf

In [6]:
what_to_ask="What was a program for animating choropleth maps was created?"

In [7]:
retrieves=retriever.invoke(what_to_ask)

In [20]:
# Get the view of top 3 chunks gotten from the DB.
retrieves

[Document(page_content='6 cartographic perspectives Number 13, Fall 1992 \nREQUIRED RESEARCH addition, a program for animating choropleth maps was created (MacChoro \nII 1989). \nAll these works employed the application of cartographic animation.', metadata={'source': 'Doc1.pdf', 'page': 6}),
 Document(page_content='II 1989). \nAll these works employed the application of cartographic animation. \nMost of them exclusively deal with the reali7ation of an animated map \nsequence for one special issue. Only the later works pick up a more', metadata={'source': 'Doc1.pdf', 'page': 6}),
 Document(page_content='well as on animation techniques and their uses. \nCampbell, C. S. and Egbert, S. L. 1990. Animated cartography: thirty \nyears of scratching the surface. Cartogmphicn. 27 (2):24-46. \nCornwell, B. and Robinson, A. 1966. Possibilities for computer ani\xad', metadata={'source': 'Doc1.pdf', 'page': 8}),
 Document(page_content="perception and production of animated map sequences is not avai

In [19]:
str_retrieves = " ".join([str(element) for element in retrieves])

In [11]:
#Instuction to be passed to the chatgpt.
instruction="Answer the user quesestion as accuractely as possible, based on the following context :"+str_retrieves

In [13]:
#Using RAPID API's free Chatgpt API to get answer based on user query.

# You can get your key from here : https://rapidapi.com/haxednet/api/chatgpt-api8
import requests

url = "https://chatgpt-api8.p.rapidapi.com/"

payload = [
	{
		"content": instruction,
		"role": "system"
	},
	{
		"content": what_to_ask,
		"role": "user"
	}
]
headers = {
	"content-type": "...",
	"X-RapidAPI-Key": "...",
	"X-RapidAPI-Host": "..."
}


response = requests.post(url, json=payload, headers=headers)

print(response.json())

{'text': 'A program called MacChoro II was created for animating choropleth maps in 1989.', 'finish_reason': 'stop', 'model': 'gpt-3.5-turbo-030', 'server': 'backup-K'}


In [17]:
print(response.json()['text'])

A program called MacChoro II was created for animating choropleth maps in 1989.
